**YOLOV5**


---

1. Preparations
2. Preprocess data
3. Model training
4. Pretrained model inference and evaluation
5. Convert to TFLite model
6. Tflite model evaluation

In [1]:
import os
import glob
import PIL
import shutil
import tqdm
import pandas as pd
import numpy as np
import IPython

###1. Preparations

In [ ]:
!pip uninstall opencv_python_headless
!pip install opencv-python-headless==4.5.4.60
!pip install fiftyone

In [ ]:
import fiftyone as fo
import fiftyone.zoo as foz

COCO-2017

In [ ]:
coco_80classes = ['person', 'bicycle', 'car', 'motorcycle', 'airplane', 'bus', 'train', 'truck', 'boat', 'traffic light',
        'fire hydrant', 'stop sign', 'parking meter', 'bench', 'bird', 'cat', 'dog', 'horse', 'sheep', 'cow',
        'elephant', 'bear', 'zebra', 'giraffe', 'backpack', 'umbrella', 'handbag', 'tie', 'suitcase', 'frisbee',
        'skis', 'snowboard', 'sports ball', 'kite', 'baseball bat', 'baseball glove', 'skateboard', 'surfboard',
        'tennis racket', 'bottle', 'wine glass', 'cup', 'fork', 'knife', 'spoon', 'bowl', 'banana', 'apple',
        'sandwich', 'orange', 'broccoli', 'carrot', 'hot dog', 'pizza', 'donut', 'cake', 'chair', 'couch',
        'potted plant', 'bed', 'dining table', 'toilet', 'tv', 'laptop', 'mouse', 'remote', 'keyboard', 'cell phone',
        'microwave', 'oven', 'toaster', 'sink', 'refrigerator', 'book', 'clock', 'vase', 'scissors', 'teddy bear',
        'hair drier', 'toothbrush']

In [ ]:
classes = ["dog","cat", "person"]

coco_train_dataset = foz.load_zoo_dataset(
    "coco-2017",
    splits=["train"],
    label_types=["detections"],
    classes=classes,
    only_matching=True,
)

coco_val_dataset = foz.load_zoo_dataset(
    "coco-2017",
    splits=["validation"],
    label_types=["detections"],
    classes=classes,
    only_matching=True,
)

coco_train_dataset.export(
   export_dir="/content/data",
   dataset_type = fo.types.dataset_types.YOLOv5Dataset,
   split='train',
   classes=coco_80classes
)

coco_val_dataset.export(
   export_dir="/content/data",
   dataset_type = fo.types.dataset_types.YOLOv5Dataset,
   split='val',
   classes=coco_80classes
)

In [ ]:
other_classes = ['person', 'horse', 'sheep', 'cow', 'elephant', 'bear', 'zebra', \
                 'giraffe', 'motorcycle', 'wine glass', 'chair', 'laptop', 'book', 'teddy bear']

other_coco_train_dataset = foz.load_zoo_dataset(
    "coco-2017",
    splits=["train"],
    label_types=["detections"],
    classes=other_classes,
    only_matching=True,
    max_samples=30000, # 20000
)

other_coco_val_dataset = foz.load_zoo_dataset(
    "coco-2017",
    splits=["validation"],
    label_types=["detections"],
    classes=other_classes,
    only_matching=True,
    max_samples=1000,
)

other_coco_train_dataset.export(
   export_dir="/content/data",
   dataset_type = fo.types.dataset_types.YOLOv5Dataset,
   split='train',
   classes=coco_80classes
)

other_coco_val_dataset.export(
   export_dir="/content/data",
   dataset_type = fo.types.dataset_types.YOLOv5Dataset,
   split='val',
   classes=coco_80classes
)

Open Images v6

In [ ]:
oiv6_train_dataset = foz.load_zoo_dataset(
    "open-images-v6",
    splits=["train"],
    label_types=["detections"],
    classes=["Dog","Cat", "Person"],
    only_matching=True,
)

oiv6_val_dataset = foz.load_zoo_dataset(
    "open-images-v6",
    splits=["validation"],
    label_types=["detections"],
    classes=["Dog","Cat", "Person"],
    only_matching=True,
)

oiv6_train_dataset.export(
   export_dir="/content/data",
   dataset_type = fo.types.dataset_types.YOLOv5Dataset,
   split='train',
   classes=[label.capitalize() for label in coco_80classes]
)

oiv6_val_dataset.export(
   export_dir="/content/data",
   dataset_type = fo.types.dataset_types.YOLOv5Dataset,
   split='val',
   classes=[label.capitalize() for label in coco_80classes]
)

In [ ]:
other_classes = ['Bear', 'Book', 'Chair', 'Elephant', 'Giraffe', 'Horse', 'Laptop', \
                 'Motorcycle', 'Person', 'Sheep', 'Teddy bear', 'Wine glass', 'Zebra']

other_oiv6_train_dataset = foz.load_zoo_dataset(
    "open-images-v6",
    splits=["train"],
    label_types=["detections"],
    classes=other_classes,
    only_matching=True,
    max_samples=30000, #20000
)

other_oiv6_val_dataset = foz.load_zoo_dataset(
    "open-images-v6",
    splits=["validation"],
    label_types=["detections"],
    classes=other_classes,
    only_matching=True,
    max_samples=2000,
)

other_oiv6_train_dataset.export(
   export_dir="/content/data",
   dataset_type = fo.types.dataset_types.YOLOv5Dataset,
   split='train',
   classes=[label.capitalize() for label in coco_80classes]
)

other_oiv6_val_dataset.export(
   export_dir="/content/data",
   dataset_type = fo.types.dataset_types.YOLOv5Dataset,
   split='val',
   classes=[label.capitalize() for label in coco_80classes]
)

In [ ]:
label_paths = glob.glob('/content/data/labels/train/*.txt')
print(len(label_paths))
instances_dict = {key: 0 for key in coco_80classes}

for label_path in label_paths:
    with open(label_path) as f:
        org_lines = f.readlines()
        org_lines = [line.rstrip() for line in org_lines]
    for line in org_lines:
        int_label = int(line.split(' ')[0])
        instances_dict[coco_80classes[int_label]] += 1
print(instances_dict)

Generate data person with pet from data fiftyone

In [ ]:
# unzip file person_pet.zip
!unzip person_pet.zip

In [ ]:
cnt = 0

# Split person_pet to train and validation
for filename in os.listdir('person_pet/images/'):
  img_path = 'person_pet/images/' + filename
  label_path = img_path.replace('images', 'labels').replace('jpg', 'txt')
  if cnt <= 11000:
    path_save_img = 'data/images/train'
    path_save_label = 'data/labels/train'
  else:
    path_save_img = 'data/images/val'
    path_save_label = 'data/labels/val'
  shutil.copy(img_path, path_save_img)
  shutil.copy(label_path, path_save_label)
  cnt += 1

Custom dataset

In [ ]:
!mkdir /content/data/custom_dataset

!gdown --id 1X4fBvBlpHMWDoEXHeZBxbjo9Cvu1K40M -O /content/data/custom_dataset/cat_data_7.zip
!gdown --id 1tkhSVoNbRM8W_P7zJgaTXoCGoi7SrRx1 -O /content/data/custom_dataset/cat_data_8.zip
!gdown --id 1MI8QF49tiUBhgwotdNbeNH5miwNv_iOx -O /content/data/custom_dataset/cat_data_9.zip
!gdown --id 1U3CfS7EqfGToDmjkYjeT9g3p-E6fUnBk -O /content/data/custom_dataset/dog_data_1.zip
!gdown --id 1TS4FvrYVj7_SjnVA_sZ8rsfONYs_XguE -O /content/data/custom_dataset/dog_data_2.zip
!gdown --id 16IkfdQx0PlWLqGV8Cx9cEbR-mUllnJvN -O /content/data/custom_dataset/dog_data_3.zip
!gdown --id 1nvt8n6ZYvMq0Cs0qs08IStl-O29ue7xn -O /content/data/custom_dataset/dog_data_4.zip
!gdown --id 1yAu1HerU5KI6GJwhm0rdRrGWk8Qv-NKp -O /content/data/custom_dataset/dog_data_5.zip
!gdown --id 1fZ8VNafJspEp38XxpR1DuhkMeLvCFrXj -O /content/data/custom_dataset/dog_data_6.zip

!unzip -qq /content/data/custom_dataset/cat_data_7.zip -d /content/data/custom_dataset/
!unzip -qq /content/data/custom_dataset/cat_data_8.zip -d /content/data/custom_dataset/
!unzip -qq /content/data/custom_dataset/cat_data_9.zip -d /content/data/custom_dataset/
!unzip -qq /content/data/custom_dataset/dog_data_1.zip -d /content/data/custom_dataset/
!unzip -qq /content/data/custom_dataset/dog_data_2.zip -d /content/data/custom_dataset/
!unzip -qq /content/data/custom_dataset/dog_data_3.zip -d /content/data/custom_dataset/
!unzip -qq /content/data/custom_dataset/dog_data_4.zip -d /content/data/custom_dataset/
!unzip -qq /content/data/custom_dataset/dog_data_5.zip -d /content/data/custom_dataset/
!unzip -qq /content/data/custom_dataset/dog_data_6.zip -d /content/data/custom_dataset/

!rm /content/data/custom_dataset/*.zip

###2. Preprocess data

**Custom dataset**

In [ ]:
def customdataset_edit_label(image_paths, target_dir):
    for image_path in tqdm.tqdm(image_paths):
        txt_path = image_path.replace('img_data', 'label_data') + '.txt'
        width, height = PIL.Image.open(image_path).size

        # read file txt
        with open(txt_path) as f:
            org_lines = f.readlines()
            org_lines = [line.rstrip() for line in org_lines]

        # edit file txt
        new_lines = []
        for line in org_lines:
            if "dog" in line:
                cls = "16"
            elif "cat" in line:
                cls = "15"
            else:
                continue
            
            old_x = int(line.split(",")[0].split(" ")[1])
            old_y = int(line.split(",")[1])
            old_w = int(line.split(",")[2])
            old_h = int(line.split(",")[3])
            new_x = ((old_x + old_w/2)/width)
            new_y = ((old_y + old_h/2)/height)
            new_w = (old_w/width)
            new_h = (old_h/height)

            new_line = str(cls) + " "+str(new_x)+ " "+str(new_y)+ " "+str(new_w)+ " "+str(new_h)
            new_lines.append(new_line)

        # write to new file txt
        txt_name = txt_path.split('/')[-1].replace('.jpg', '')
        new_txt_path = os.path.join(target_dir, txt_name)
        print(f'write to {new_txt_path}')
        with open(new_txt_path, "w") as fobj:
            fobj.write("\n".join(new_lines))

In [ ]:
custom_file_paths = glob.glob('/content/data/custom_dataset/*/*.jpg')
print(len(custom_file_paths))
print(custom_file_paths[:2])

# split VAL_SET
import random
random.seed(2021)

random.shuffle(custom_file_paths)
TRAIN_CUSTOM_FILEPATHS = custom_file_paths[:50000]
VAL_CUSTOM_FILEPATHS = custom_file_paths[50000:]

print(len(TRAIN_CUSTOM_FILEPATHS), len(VAL_CUSTOM_FILEPATHS))

In [ ]:
customdataset_edit_label(TRAIN_CUSTOM_FILEPATHS, '/content/data/labels/train')
customdataset_edit_label(VAL_CUSTOM_FILEPATHS, '/content/data/labels/val')

In [ ]:
for file_path in tqdm.tqdm(TRAIN_CUSTOM_FILEPATHS):
    file_name = file_path.split('/')[-1]
    destination = os.path.join('/content/data/images/train', file_name)
    # move only files
    if os.path.isfile(file_path):
        shutil.move(file_path, destination)

for file_path in tqdm.tqdm(VAL_CUSTOM_FILEPATHS):
    file_name = file_path.split('/')[-1]
    destination = os.path.join('/content/data/images/val', file_name)
    # move only files
    if os.path.isfile(file_path):
        shutil.move(file_path, destination)

In [ ]:
print(len(os.listdir('/content/data/labels/train')))
print(len(os.listdir('/content/data/labels/val')))
print(len(os.listdir('/content/data/images/train')))
print(len(os.listdir('/content/data/images/val')))

###**3. Model training**

Download the code in github

In [ ]:
# clone YOLOv5
if not os.path.isdir("yolov5"):
    !git clone https://github.com/ultralytics/yolov5  # clone repo
    %cd yolov5
    %pip install -qr requirements.txt # install dependencies
else:
    %cd yolov5
    
import torch
import os
from IPython.display import Image, clear_output  # to display images

print(f"Setup complete. Using torch {torch.__version__} ({torch.cuda.get_device_properties(0).name if torch.cuda.is_available() else 'CPU'})")

**Create the config YAML file**

In [ ]:
CONFIG_DIR = '/content/yolov5/data'
CONFIG_FILE = os.path.join(CONFIG_DIR, "pet.yaml")
if not os.path.exists(CONFIG_DIR):
    os.mkdir(CONFIG_DIR)

config_text = \
"""path: /content/data/images
train: train
val: val
test:

nc: 80  # number of classes
names: ['person', 'bicycle', 'car', 'motorcycle', 'airplane', 'bus', 'train', 'truck', 'boat', 'traffic light',
        'fire hydrant', 'stop sign', 'parking meter', 'bench', 'bird', 'cat', 'dog', 'horse', 'sheep', 'cow',
        'elephant', 'bear', 'zebra', 'giraffe', 'backpack', 'umbrella', 'handbag', 'tie', 'suitcase', 'frisbee',
        'skis', 'snowboard', 'sports ball', 'kite', 'baseball bat', 'baseball glove', 'skateboard', 'surfboard',
        'tennis racket', 'bottle', 'wine glass', 'cup', 'fork', 'knife', 'spoon', 'bowl', 'banana', 'apple',
        'sandwich', 'orange', 'broccoli', 'carrot', 'hot dog', 'pizza', 'donut', 'cake', 'chair', 'couch',
        'potted plant', 'bed', 'dining table', 'toilet', 'tv', 'laptop', 'mouse', 'remote', 'keyboard', 'cell phone',
        'microwave', 'oven', 'toaster', 'sink', 'refrigerator', 'book', 'clock', 'vase', 'scissors', 'teddy bear',
        'hair drier', 'toothbrush']  # class names
"""

with open(CONFIG_FILE, "w") as fwrite:
    fwrite.write(config_text)

In [ ]:
!python train.py \
    --img 320 \
    --weights /content/last.pt \
    --epochs 100 \
    --data pet.yaml \
    --save-period 30 \
    --project model_new \
    --name exp \
    --exist-ok

##Inference

In [ ]:
!python3 yolov5/detect.py \
    --weights /content/best.pt \
    --img 320 \
    --source "/content/test_pet_detection/*.jpg"\
    --conf-thres 0.3 \
    --save-crop \
    --classes 15 16

**Convert to TFLite model**

In [ ]:
!python export.py \
  --weights /content/weights/best.pt\
  --include tflite \
  --data /content/yolov5/data/pet.yaml \
  --imgsz 320